# Connect to Asana instance

In [1]:
import os
import asana
from pathlib import Path
import datetime

# @icaoberg Read secrets. Assumes file exists in ~/.ASANA_SECRETS
file = str(Path.home()) + os.sep + '.ASANA_SECRETS'
exec(open(file).read())

client = asana.Client.access_token(PERSONAL_ACCESS_TOKEN)

## Information and parameters

In [2]:
#SOFTWARE THAT I WANT TO INSTALL
software='raxml'
version='8.2.12'

username='1183587634011025' #icaoberg
workspace='1132331877475002' #PSC
team='1192784280891409' #NCGAS

maindelta=7
when=datetime.date.today() + datetime.timedelta(days=maindelta+1)

# Create project

In [3]:
params = {'name':'Install ' + software + ' version ' + version, 
          'color':'light-red', \
          'archived':False, \
          'is_template':False, \
          'public': False, \
          'notes':'Software that needs to be tested and installed on Bridges/Bridges2.', \
          'default_view':'calendar', \
          'due_on':str(when + datetime.timedelta(days=7)), \
          'start_on':str(when), \
          'created_by':client.users.get_user(username)['name']}

answer = client.projects.create_in_team( team, params )
project = answer['gid']

# Create tasks

In [4]:
strings = { 'Check if the latest version of the software is hosted on Biocontainers as a Singularity image', \
            'Create repository on GitLab to host recipes and scripts', \
            'Create or find the Singularity recipe', \
            'Create script for each binary in library/package', \
            'Create a modulefile',
            'Pull or build Singularity on testing server', \
            'Find data to test the software and a script to test the image', 
            'Add recipe, modulefile and scripts to repository in GitLab',
            'Clone or copy to pylon5 on Bridges', 
            'Install modulefile locally and test Singularity image', \
            'Add Singularity image and scripts to /opt/packages', \
            'Add modulefile to modules repository and create a pull request',
            'Add modulefile to /opt/modules', \
            'Test one last time if needed' }

deltas=[1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7]

for delta, string in zip(deltas, strings):
    params={'name': string, \
            'assignee': username, \
            'projects': [project], \
            'due_on': str(when+datetime.timedelta(days=maindelta+1+delta)), \
            'start_on': str(when)}

    answer = client.tasks.create_in_workspace(workspace,params)

# Rename default section

In [5]:
answer = client.sections.find_by_project(project)
section = next(answer)
sectionid = section['gid']
params = {'name':'To Do'}

answer = client.sections.update( sectionid, params )

# Create sections

In [6]:
strings = { 'In Progress', \
           'Ready to test/testing on Bridges', \
           'Ready to test/testing on Bridges2', \
           'Done'}

for string in strings:
    params={'name': string}
    answer = client.sections.create_in_project(project, params)